<a href="https://colab.research.google.com/github/rb58853/ML-RSI-Images/blob/main/code/CLIP-Embedding/CLIPEmbedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!git clone https://github.com/rb58853/ML-RSI-Images.git

Cloning into 'ML-RSI-Images'...
remote: Enumerating objects: 681, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 681 (delta 5), reused 17 (delta 4), pack-reused 660
Receiving objects: 100% (681/681), 22.61 MiB | 30.71 MiB/s, done.
Resolving deltas: 100% (262/262), done.


In [7]:
import os
os.chdir('ML-RSI-Images/code/CLIP-Embedding')

In [8]:
from clip_embeding import ClipEmbedding
clip = ClipEmbedding()

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [10]:
from PIL import Image
image_path = '../../images/'
image = Image.open(image_path+'image_4.jpg')


In [11]:
text = 'two dogs in a white and black couch'

In [12]:
image_embedding = clip.get_image_embedding(image)
text_embedding = clip.get_text_embedding(text)
print(clip.calculate_similarity(image_embedding, text_embedding))

0.32101574540138245
